In [2]:
import os
from pytube import YouTube
import subprocess

In [3]:
link = 'https://youtu.be/tdsEmYy7rOM'

name = 'test'

resolution="1440"

path = "d:\\youtube"

yt = YouTube(link)
videos = yt.streams

In [4]:
video = {}
audio = {}
for index, item in (enumerate(videos)):
    item = str(item)
    if resolution in item:
        if 'mp4' in item:
            video['mp4'] = index   
        else:
            video['webm'] = index
    if 'audio' in item:
        if 'mp4' in item:
            item = item.split(' ')
            abr = int(item[3][5:-5])
            if not 'mp4' in audio:
                audio['mp4'] = {
                    'id':index,
                    'abr':abr
                } 
            elif audio['mp4']['abr'] < abr:
                audio['mp4'] = {
                    'id':index,
                    'abr':abr
                }       
        else:
            item = item.split(' ')
            abr = int(item[3][5:-5])
            if not 'webm' in audio:
                audio['webm'] = {
                    'id':index,
                    'abr':abr
                } 
            elif audio['webm']['abr'] < abr:
                audio['webm'] = {
                    'id':index,
                    'abr':abr
                }
if 'mp4' in video:
    vdict = {
        'res':'mp4',
        'video_id': video['mp4'],
        'audio_id': audio['mp4']['id']
        }
else:
    vdict = {
        'res':'webm',
        'video_id': video['webm'],
        'audio_id': audio['webm']['id']
        }

print(vdict)

{'res': 'mp4', 'video_id': 13, 'audio_id': 18}


In [21]:
video_print = videos[vdict['video_id']].download(output_path=path, filename='_video.' + vdict['res'])
print(video_print)
audio_print = videos[vdict['audio_id']].download(output_path=path, filename='_audio.' + vdict['res'])
print(audio_print)

d:\youtube\_video.mp4
d:\youtube\_audio.mp4


In [7]:
args = [
    'ffmpeg',
    '-i',
    os.path.join(path, "_video."+ vdict['res']),
    '-i',
    os.path.join(path, "_audio."  + vdict['res']),
    '-c',
    'copy',
    str(os.path.join(path, name + "." + vdict['res']))
]
subprocess.run(args)


['ffmpeg', '-i', 'd:\\youtube\\_video.mp4', '-i', 'd:\\youtube\\_audio.mp4', '-c', 'copy', 'd:\\youtube\\test.mp4']


CompletedProcess(args=['ffmpeg', '-i', 'd:\\youtube\\_video.mp4', '-i', 'd:\\youtube\\_audio.mp4', '-c', 'copy', 'd:\\youtube\\test.mp4'], returncode=0)

In [8]:
# Удаляем файлы
os.remove(os.path.join(path, "_video." + vdict['res']))
os.remove(os.path.join(path, "_audio." + vdict['res']))